In [1]:
from transformers import AutoModelForCausalLM 
from transformers import AutoProcessor 

model_id = "microsoft/Phi-3-vision-128k-instruct" 

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", trust_remote_code=True, torch_dtype="auto")

# model = AutoModelForCausalLM.from_pretrained('microsoft/Phi-3-vision-128k-instruct', device_map="cuda", trust_remote_code=True, torch_dtype="auto", _attn_implementation="eager")

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True) 


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
from PIL import Image 

messages = [ 
    {"role": "system", "content": "You are a helpful assistant."}, 
    {"role": "user", "content": "<|image_1|>\nWhat is shown in this image?"}
] 


image_url = "dogs-playing-in-grassy-field.jpg"
image = Image.open(image_url) 

prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = processor(prompt, image, return_tensors="pt").to("cuda:0") 

print(f">>> Prompt\n{prompt}")

>>> Prompt
<|system|>
You are a helpful assistant.<|end|>
<|user|>
<|image_1|>
What is shown in this image?<|end|>
<|assistant|>



In [ ]:
generation_args = { 
    "max_new_tokens": 1000, 
    "temperature": 0.2, 
    "do_sample": True, 
} 

generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) 


generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, 
                                  skip_special_tokens=True, 
                                  clean_up_tokenization_spaces=False)[0]
print(f'>>> Response\n{response}')

In [20]:
import torch
torch.cuda.empty_cache()

In [4]:
import json
import requests
from PIL import Image 
from io import BytesIO
image = Image.open("dogs-playing-in-grassy-field.jpg").convert("RGB")

server_url = "http://127.0.0.1:8000/predict"

# Convert the image to bytes
buffered = BytesIO()
image.save(buffered, format="JPEG")
image_bytes = buffered.getvalue()
messages = [ 
    {"role": "system", "content": "You are a helpful assistant."}, 
    {"role": "user", "content": "<|image_1|>\nWhat is shown in this image?"}
] 

data = {
        "images": [
            {"image_bytes":image_bytes.hex()}, 
        ],
        "messages": messages,
    }
# Send a POST request to the server with the image
response = requests.post(
    server_url,
    json= json.dumps(data)
)
response.json()

{'output': 'The image displays an abbreviated tax invoice from Saathimart.com for Khileshworl Trading Pvt. Ltd. located in Narephat, Kathmandu. The VAT number is 610282856. The invoice is for Bill No. S11992-NKT-80/81, dated 01/29/2024, with the name Miti and address PAN No. The payment mode is cash. The invoice lists five items: CURRENT MIXED, EVEREST, EVEREST GARAM, EVEREST CHICKEN, and DHARA SUNFLOWER, with their respective quantities, rates, and amounts. The gross amount is 780.00, with a discount of 0.00, resulting in a net amount of 780.00. The tender is also 780.00, and there is no change. The amount to be paid is Rs. Seven hundred eighty only. Conditions apply, including that goods once sold will not be returned and that the exchange must be within 7 days with receipt except on Saturday. The invoice thanks the customer for shopping with Saathimart and provides a download link for the Saathimart app, along with a support contact number. The invoice is signed by the cashier at 10